In [11]:
!pip install opencv-python numpy
import cv2
print(cv2.__version__)
import numpy as np
import requests
import time

4.9.0



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
def setup_camera():
    # Initialize the camera
    camera = cv2.VideoCapture(0)  # Adjust the index if needed for your camera
    if not camera.isOpened():
        raise Exception("Could not open camera.")
    return camera

In [13]:
def detect_dots(frame, color_ranges):
    """
    Detect dots of a specific color in the frame.
    Args:
        frame: Input image (BGR format).
        color_ranges: List of (lower, upper) HSV bound tuples for the color.
    Returns:
        List of detected dot centers [(x, y), ...].
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    combined_mask = None

    # Create a combined mask for all provided ranges
    for lower, upper in color_ranges:
        mask = cv2.inRange(hsv, lower, upper)
        if combined_mask is None:
            combined_mask = mask
        else:
            combined_mask = cv2.bitwise_or(combined_mask, mask)

    # Apply morphological operations to clean up noise
    # combined_mask = cv2.erode(combined_mask, None, iterations=1)
    combined_mask = cv2.dilate(combined_mask, None, iterations=2)

    # Find contours of the detected areas
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    centers = []
    for contour in contours:
        if cv2.contourArea(contour) > 10:  # Ignore very small areas
            M = cv2.moments(contour)
            if M["m00"] > 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                centers.append((cX, cY))

    # Show mask
    cv2.imshow("Mask", combined_mask)

    print(len(centers), "dots detected")
    return centers

In [14]:
def draw_circle(image, center, radius, color, thickness=2):
    """
    Draw a circle on an image.
    """
    cv2.circle(image, center, radius, color, thickness)

In [15]:
def generate_circle_positions(radius, center, num_points):
    """
    Generate positions on a circle.
    Args:
        radius: Radius of the circle in mm.
        center: Tuple (x, y) for the circle's center.
        num_points: Number of positions to generate.
    Returns:
        List of (x, y) tuples.
    """
    positions = []
    for i in range(num_points):
        angle = 2 * np.pi * i / num_points
        x = center[0] + radius * np.cos(angle)
        y = center[1] + radius * np.sin(angle)
        positions.append((int(x), int(y)))
    return positions

In [16]:
def send_motion_command(ip_address, position):
    """
    Send motion command to the robot.
    Args:
        ip_address: IP address of the robot.
        position: Tuple (x, y) for the target position.
    """
    pos_str = '{"cmd":"motion","speed":5,"nosplit":1,"pos":[{"a":0,"p":' + str(position[0]) + '},{"a":1,"p":' + str(position[1]) + '}]}'

    url = f'http://{ip_address}/api/motors/json/{pos_str}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(f"Command sent successfully: {position}")
        else:
            print(f"Failed to send command. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error sending motion command: {e}")

In [17]:
# Define HSV color ranges for red and blue dots - use PickDotColours.py to find these values
red_ranges = [
    (np.array([0, 150, 140]), np.array([15, 255, 255])),    # Lower red range
    (np.array([145, 50, 50]), np.array([179, 255, 255])) # Upper red range
]
blue_ranges = [
    (np.array([90, 190, 150]), np.array([110, 235, 255]))  # Single range for blue
]

In [18]:
camera = setup_camera()

In [19]:
# Circle parameters
radius = 180  # in mm
center = (0, 0)
num_positions = 8
circle_positions = generate_circle_positions(radius, center, num_positions)

# Robot parameters
robot_ip = "192.168.86.221"

# Motion timing
last_motion_time = time.time()
motion_interval = 5  # seconds
position_index = 0

In [20]:
# Main loop
center_point = None
blue_centers = []
radius = 0
while True:
    ret, frame = camera.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    # Detect blue dots initially
    if center_point is None:
        blue_centers = detect_dots(frame, blue_ranges)

        if len(blue_centers) >= 4:

            # Calculate the center of the circle
            center = np.mean(blue_centers, axis=0).astype(int)
            center_point = (center[0], center[1])

            # Calculate radius as the average distance to the blue dots
            radius = int(np.mean([np.linalg.norm(np.array(center) - np.array(pt)) for pt in blue_centers]))
    else:
        # Draw the large circle and center point
        for pt in blue_centers:
            draw_circle(frame, pt, 5, (255, 0, 0), -1)  # Blue dots
        draw_circle(frame, center_point, 5, (0, 255, 0), -1)  # Green center
        draw_circle(frame, center_point, radius, (255, 255, 255), 2)  # Large circle

        # Detect red dot (end-effector)
        red_centers = detect_dots(frame, red_ranges)
        for red_center in red_centers:
            cv2.drawMarker(frame, red_center, (0, 0, 255), cv2.MARKER_CROSS, 10, 2)  # Red cross

        # Check if the red dot is within the circle
        for red_center in red_centers:
            distance = np.linalg.norm(np.array(center_point) - np.array(red_center))
            if distance < (radius - 15):
                print("WARNING: Red dot is too close to the circle!")

    # Send motion commands every 5 seconds
    current_time = time.time()
    if current_time - last_motion_time >= motion_interval:
        position = circle_positions[position_index]
        send_motion_command(robot_ip, position)
        position_index = (position_index + 1) % len(circle_positions)
        last_motion_time = current_time

    # Show the output frame
    cv2.imshow("Robot Arm Tracking", frame)

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release camera and close windows
camera.release()
cv2.destroyAllWindows()

3 dots detected
3 dots detected
3 dots detected
4 dots detected
1 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
1 dots detected
0 dots detected
1 dots detected
0 dots detected
1 dots detected
0 dots detected
0 dots detected
0 dots detected
1 dots detected
1 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
0 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots detected
0 dots detected
0 dots detected
1 dots detected
1 dots detected
1 dots detected
1 dots d